In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import norm

In [2]:
data = pd.read_csv('usdjpy.csv', decimal='.')
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
data['Rendimientos'] = np.log(data['USD por 1 JPY'] / data['USD por 1 JPY'].shift(1))


In [21]:
def backtesting_delta_normal(data, monto, nc, posicion, n_muestras):

    medias, volatilidades, VaR, CVaR, fechas = [], [], [], [], []
    perdidas = []
    excedentes = []

    monto = float(monto)
    nc = float(nc)
    posicion = posicion.lower()
    n_muestras = int(n_muestras)

    if 'Date' not in data.columns:
        print('No hay datos. Vuelve a intentar con DF de columnas "Date" en orden ascendente y el precio del activo solamente.')
        return

    if monto < 0 or monto > 100000000:
        print("Monto fuera de rango (debe estar entre 0 y 100,000,000). Vuelve a intentar")
        return

    if nc < 0.01 or nc > 0.99:
        print("Nivel de confianza fuera de rango (debe estar entre 0.01 y 0.99). Vuelve a intentar")
        return

    if posicion not in ["long", "larga", "l", "short", "corta", "s", "c"]:
        print('Posición incorrecta. Vuelve a intentar')
        return

    if n_muestras < 0 or n_muestras > len(data)/2 - 1:
        print("Número de muestras inválido, debes ingresar un número positivo menor a la mitad de datos totales")
        return

    z = norm.ppf(1 - nc)
    fi = norm.pdf(z)

    muestras_df = pd.DataFrame()
    muestras_df = data.tail(n_muestras*2)

    for i in range(n_muestras):
        rango = data.iloc[0+i:-(n_muestras-i)]
        medias.append(rango.Rendimientos.mean())
        volatilidades.append(rango.Rendimientos.std(ddof=1))
        VaR.append(abs((medias[i] + z * volatilidades[i]) * monto))
        CVaR.append(((medias[i] + fi) / (1 - nc) * volatilidades[i])*monto)

    results_df = pd.DataFrame(index=range(n_muestras))
    results_df['medias'] = medias
    results_df['volatilidades'] = volatilidades
    results_df['VaRs'] = VaR
    results_df['CVaR'] = CVaR

    return results_df


In [22]:
r = backtesting_delta_normal(data, 50000, 0.90, 'short', 60)
r

,medias,volatilidades,VaRs,CVaR
0,-0.000388,0.005962,401.459355,522.010667
1,-0.000388,0.005959,401.246633,521.752783
2,-0.000382,0.005962,401.152033,522.026569
3,-0.000402,0.005990,403.965811,524.449578
4,-0.000399,0.005994,404.018604,524.740256
5,-0.000394,0.005993,403.739566,524.726770
6,-0.000390,0.005990,403.355218,524.461230
7,-0.000395,0.005994,403.837323,524.783267
8,-0.000400,0.005996,404.181409,524.923387
9,-0.000410,0.005998,404.817029,525.060189
